In [1]:
import sys

BASE_DIR = '../'
sys.path.insert(0, BASE_DIR)

from src import Preprocessor, Logger, KMeansConnector

import configparser
from pyspark.sql import SparkSession

CONFIG_PATH = '../config.ini'

In [2]:
config = configparser.ConfigParser()
config.read(CONFIG_PATH)

['../config.ini']

In [3]:
spark = SparkSession.builder.appName(config['spark']['app_name']) \
    .master(config['spark']['deploy_mode']) \
    .config("spark.driver.host", config['spark']['host'])\
    .config("spark.driver.bindAddress", config['spark']['host']) \
    .config("spark.driver.cores", config['spark']['driver_cores']) \
    .config("spark.executor.cores", config['spark']['executor_cores']) \
    .config("spark.driver.memory", config['spark']['driver_memory']) \
    .config("spark.executor.memory", config['spark']['executor_memory']) \
    .getOrCreate()

24/09/16 18:30:42 WARN Utils: Your hostname, tower resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface enp4s0)
24/09/16 18:30:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/16 18:30:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
logger = Logger(True)
preproc = Preprocessor(logger)

In [5]:
features_names = config['dataset']['features'].split(", ")
output = preproc.load(f"../data/{config['dataset']['name']}", spark, features_names)
scaled_output = preproc.apply_scale(output)

In [10]:
scaled_output = scaled_output.collect()

In [12]:
scaled_output[0].scaled_features

DenseVector([0.0, 23.8275, 25.2062, 1.8355])

In [6]:
model = KMeansConnector(2, 'scaled_features', logger)

In [7]:
model.fit(scaled_output)

In [9]:
predictions = model.predict(scaled_output)

In [11]:
scores = model.evaluate(predictions)

In [13]:
scores

0.6272703958697375